In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
VWAP 3-Candle Intraday Backtester (NSE, 5-min)
==============================================

Strategy Overview
-----------------
Implements the **VWAP 3-candle pattern** using a single VWAP line:

Definitions
-----------
- **Opening Candle**  : first candle that **closes above** (for long) or **below** (for short) the VWAP line.
- **Signal Candle**   : the next candle that **breaks the high/low** of the Opening Candle in the same direction.
- **Entry Candle**    : the next candle that **breaks the Signal Candle’s high/low** (directional confirmation).
- **Invalidation**    : if, instead of breaking in the intended direction, the Entry candidate breaks the
                        opposite side of the Signal Candle first, the setup is discarded.

Execution Model
---------------
- **Entry** executes when the Entry Candle breaks the trigger:
  • LONG  : trigger = Signal High; fill price = max(current bar OPEN, trigger)
  • SHORT : trigger = Signal Low ; fill price = min(current bar OPEN, trigger)
- **One trade per day per stock** (first valid setup only).
- **Optional proximity filter**: require entry to be reasonably close to VWAP
  (`max_vwap_gap_pct`, e.g., 0.5% from VWAP at entry bar).

Risk Management
---------------
- Fixed absolute rupee SL/TP per **position** (converted per-share using position size):
  • Stop Loss (SL): `sl_rupees`
  • Take Profit (TP): `tp_rupees`
- **Trailing SL (optional)** (`enable_trailing_sl`):
  • For LONG  : trail = max(prev_trail, highest_since_entry - trail_per_share)
  • For SHORT : trail = min(prev_trail, lowest_since_entry  + trail_per_share)
  • Exit reason is “SL-TRAIL” if triggered.
- Square-off all open positions near **15:25 IST** (configurable).

Costs (Groww Intraday – NSE)
----------------------------
Includes brokerage, STT, stamp duty, exchange, SEBI, IPFT, and GST:
- Brokerage: min(₹20, 0.1% of turnover), floor ₹5 each side
- STT: 0.025% on sell turnover
- Stamp Duty: 0.003% on buy turnover
- Exchange Txn: 0.00297% each side
- SEBI + IPFT: 0.0001% each side
- GST: 18% on (brokerage + exchange + SEBI + IPFT)

Outputs
-------
- Saves trades to `trades.csv`:
  date, ticker, direction, entry/exit, qty, prices, P&L, charges, reason, turnovers, etc.
- Prints concise **overall**, **by-ticker**, and **by-direction** summaries (incl. net P&L and max drawdown).

Example Configuration
---------------------
CFG = Config(
    tickers=["RELIANCE.NS", "TCS.NS", "INFY.NS"],
    start_date="2025-10-01",
    end_date="2025-11-01",
    interval="5m",
    capital_per_stock=100_000.0,
    intraday_leverage=5.0,
    sl_rupees=3_000.0,
    tp_rupees=11_000.0,
    enable_longs=True,
    enable_shorts=False,
    enable_trailing_sl=False,
    trail_rupees=500.0,
    max_vwap_gap_pct=0.5,     # reject entries >0.5% away from VWAP
)
"""

import math
import sys
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import pandas as pd
import numpy as np

try:
    import yfinance as yf
except Exception:
    print("Please: pip install yfinance pandas numpy")
    sys.exit(1)

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str]

    start_date: str = "2025-10-01"
    end_date:   str = "2025-11-01"
    interval:   str = "5m"

    capital_per_stock: float = 100_000.0    # cash allocated per symbol
    intraday_leverage: float = 5.0          # 5× buying power

    sl_rupees: float = 2000.0               # absolute ₹ stop per position
    tp_rupees: float = 6000.0              # absolute ₹ target per position

    # Direction toggles
    enable_longs: bool = True
    enable_shorts: bool = True

    # Trailing SL
    enable_trailing_sl: bool = False
    trail_rupees: float = 500.0

    # Trading session (IST)
    entry_start: str = "10:00"              # allow setups after 10:00
    entry_end:   str = "14:00"              # last time to open a trade
    squareoff_time: str = "14:00"           # force exit if still open

    # Proximity to VWAP filter (% from VWAP at ENTRY bar)
    max_vwap_gap_pct: float = 0.5           # 0.5% default; set None/<=0 to disable

    timezone: str = "Asia/Kolkata"
    out_file: str = "trades.csv"


CFG = Config(
    tickers=['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'APLLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'AGARWALEYE.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'DUMMYSKFIN.NS', 'DUMMYTATAM.NS', 'DUMMYDBRLT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITCHOTELS.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KPITTECH.NS', 'KSB.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'THELEELA.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'MGL.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'ONESOURCE.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PGHH.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RPOWER.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'ENRIN.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TMPV.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VENTIVE.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']

)

# =========================
# GROWW CHARGES (NSE intraday)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    """
    Compute charges for a single round-trip intraday equity trade (NSE).
    """
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5

    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)

    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover

    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover

    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover

    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)

    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover

    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)

    return {
        "brokerage_buy": bro_buy, "brokerage_sell": bro_sell,
        "exchange_buy": exch_buy, "exchange_sell": exch_sell,
        "sebi_buy": sebi_buy, "sebi_sell": sebi_sell,
        "ipft_buy": ipft_buy, "ipft_sell": ipft_sell,
        "gst_buy": gst_buy, "gst_sell": gst_sell,
        "stt_sell": stt_sell, "stamp_buy": stamp_buy,
        "total_charges": total
    }

# =========================
# DATA
# =========================
def fetch_5m_dataframe(ticker: str, start: str, end: str, interval: str, tz: str) -> pd.DataFrame:
    df = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        prepost=False,
        progress=False,
        multi_level_index=False,
        group_by="column",
    )
    if df.empty:
        return df

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([c for c in col if c]).strip() for col in df.columns.values]

    # Ensure tz-aware IST
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(tz)
    else:
        df = df.tz_convert(tz)

    cols_map = {c: c.title() for c in df.columns}
    df.rename(columns=cols_map, inplace=True)
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep]
    df["Ticker"] = ticker
    return df


def compute_vwap_intraday(df: pd.DataFrame, tz: str) -> pd.DataFrame:
    """
    Compute session-reset VWAP: cum(sum(price*vol))/cum(sum(vol))
    price = typical price = (H+L+C)/3
    """
    if df.empty:
        return df.copy()

    work = df.copy()
    # session key = date (IST)
    dates = work.index.tz_convert(tz).date
    work["session_date"] = dates
    tp = (work["High"] + work["Low"] + work["Close"]) / 3.0
    pv = tp * work["Volume"].fillna(0)

    work["cum_pv"] = pv.groupby(work["session_date"]).cumsum()
    work["cum_v"]  = work["Volume"].fillna(0).groupby(work["session_date"]).cumsum()
    work["VWAP"]   = np.where(work["cum_v"] > 0, work["cum_pv"] / work["cum_v"], np.nan)

    return work.drop(columns=["cum_pv", "cum_v"])


def session_filter(df: pd.DataFrame, date: pd.Timestamp, tz: str) -> pd.DataFrame:
    day = pd.Timestamp(date)
    day_start = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 09:15").tz_localize(tz)
    day_end   = pd.Timestamp(day.date().strftime("%Y-%m-%d") + " 15:30").tz_localize(tz)
    return df.loc[(df.index >= day_start) & (df.index <= day_end)].copy()


def _ts_on_day(day: pd.Timestamp, hhmm: str, tz: str) -> pd.Timestamp:
    return pd.Timestamp(day.date().strftime("%Y-%m-%d") + f" {hhmm}").tz_localize(tz)


# =========================
# VWAP 3-CANDLE LOGIC
# =========================
def _opening_candle_condition(row: pd.Series) -> Optional[str]:
    """Return 'long' if Close>VWAP, 'short' if Close<VWAP, else None."""
    if not np.isfinite(row.get("VWAP", np.nan)):
        return None
    if row["Close"] > row["VWAP"]:
        return "long"
    if row["Close"] < row["VWAP"]:
        return "short"
    return None


def find_vwap_entry(day_df: pd.DataFrame,
                    entry_start: str,
                    tz: str,
                    enable_longs: bool,
                    enable_shorts: bool,
                    max_vwap_gap_pct: Optional[float]) -> Tuple[Optional[str], Optional[pd.Timestamp], float]:
    """
    Scan from entry_start to find the **first** valid 3-candle VWAP setup.
    Returns (direction, entry_ts, entry_price) or (None, None, nan).
    """

    start_ts = _ts_on_day(day_df.index[0], entry_start, tz)
    scan = day_df.loc[day_df.index >= start_ts]
    if len(scan) < 3:
        return None, None, np.nan

    # Iterate in rolling windows of 3 consecutive candles
    # i = Opening, i+1 = Signal candidate (must break Opening), i+2..N = Entry candidate (must break Signal)
    idx = list(scan.index)
    for i in range(len(idx) - 2):
        i0, i1 = idx[i], idx[i+1]
        open_row  = scan.loc[i0]
        sig_row   = scan.loc[i1]

        opening_dir = _opening_candle_condition(open_row)
        if opening_dir is None:
            continue
        if opening_dir == "long" and not enable_longs:
            continue
        if opening_dir == "short" and not enable_shorts:
            continue

        # Signal candle must break Opening candle's extreme in the same direction
        if opening_dir == "long":
            if sig_row["High"] <= open_row["High"]:
                continue
            trigger = float(sig_row["High"])
        else:
            if sig_row["Low"] >= open_row["Low"]:
                continue
            trigger = float(sig_row["Low"])

        # Now search forward for Entry: a candle that breaks the Signal extreme in same direction.
        # Invalidation: if the opposite side of Signal is broken first, discard the setup.
        sig_high, sig_low = float(sig_row["High"]), float(sig_row["Low"])

        for j in range(i+2, len(idx)):
            ts = idx[j]
            row = scan.loc[ts]

            invalidated = (row["Low"] < sig_low) if opening_dir == "long" else (row["High"] > sig_high)
            if invalidated:
                break  # discard this setup, move to next i

            crossed = (row["High"] >= trigger) if opening_dir == "long" else (row["Low"] <= trigger)
            if not crossed:
                continue

            # Execution price: next bar open vs trigger on the same bar = conservative fill
            if opening_dir == "long":
                entry_price = float(max(row["Open"], trigger))
            else:
                entry_price = float(min(row["Open"], trigger))

            # Optional proximity filter to VWAP at entry bar
            if max_vwap_gap_pct and max_vwap_gap_pct > 0:
                vwap_here = float(row.get("VWAP", np.nan))
                if np.isfinite(vwap_here) and vwap_here > 0:
                    gap_pct = abs(entry_price - vwap_here) / vwap_here * 100.0
                    if gap_pct > max_vwap_gap_pct:
                        # too far from VWAP → skip this entry, continue searching this setup
                        continue

            return opening_dir, ts, entry_price

    return None, None, np.nan


# =========================
# SIMULATION (with trailing SL)
# =========================
def simulate_trade(day_df: pd.DataFrame, direction: str, entry_ts: pd.Timestamp, entry_price: float,
                   capital: float, leverage: float, sl_rupees: float, tp_rupees: float,
                   squareoff_time: str, tz: str, ticker: str,
                   enable_trailing_sl: bool, trail_rupees: float):
    if math.isnan(entry_price) or entry_ts is None:
        return None

    # Position sizing with intraday leverage
    buying_power = capital * leverage
    qty = int(buying_power // entry_price)
    if qty < 1:
        return None

    # Per-share distances
    risk_per_share   = sl_rupees / qty
    target_per_share = tp_rupees / qty
    trail_per_share  = (trail_rupees if trail_rupees is not None else sl_rupees) / qty

    # Static initial stop and target
    if direction == "long":
        static_sl = entry_price - risk_per_share
        tp = entry_price + target_per_share
        high_water = entry_price  # for trailing
        dyn_sl = static_sl
    else:
        static_sl = entry_price + risk_per_share
        tp = entry_price - target_per_share
        low_water = entry_price   # for trailing
        dyn_sl = static_sl

    exit_reason = None
    exit_ts = None
    exit_price = None

    after = day_df.loc[day_df.index >= entry_ts]
    for ts, row in after.iterrows():
        # Update trailing stop BEFORE checks (ratchet only)
        if enable_trailing_sl:
            if direction == "long":
                high_water = max(high_water, float(row["High"]))
                new_sl = max(static_sl, high_water - trail_per_share)
                dyn_sl = max(dyn_sl, new_sl)
            else:
                low_water = min(low_water, float(row["Low"]))
                new_sl = min(static_sl, low_water + trail_per_share)
                dyn_sl = min(dyn_sl, new_sl)

        # Exit logic (adverse first)
        if direction == "long":
            if row["Low"] <= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            if row["High"] >= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break
        else:
            if row["High"] >= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL",
                                                    ts, float(dyn_sl))
                break
            if row["Low"] <= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break

        # EOD square-off
        so = _ts_on_day(ts, squareoff_time, tz)
        if ts >= so:
            exit_reason, exit_ts, exit_price = "EOD", ts, float(row["Close"])
            break

    if exit_ts is None:
        # If still open by last bar, exit at last close
        last_ts = after.index[-1]
        exit_reason, exit_ts, exit_price = "EOD", last_ts, float(after.iloc[-1]["Close"])

    # P&L before charges + turnovers for fees
    if direction == "long":
        gross = (exit_price - entry_price) * qty
        buy_turnover  = entry_price * qty
        sell_turnover = exit_price * qty
    else:
        gross = (entry_price - exit_price) * qty
        buy_turnover  = exit_price * qty
        sell_turnover = entry_price * qty

    fees = groww_intraday_charges(buy_turnover, sell_turnover)
    net = gross - fees["total_charges"]

    return {
        "date": entry_ts.date().isoformat(),
        "ticker": ticker,
        "direction": direction,
        "entry_time": entry_ts.isoformat(),
        "entry_price": round(entry_price, 2),
        "qty": qty,
        "sl_price_initial": round(static_sl, 2),
        "tp_price": round(tp, 2),
        "sl_trailing_enabled": enable_trailing_sl,
        "trail_rupees": trail_rupees,
        "exit_time": exit_ts.isoformat(),
        "exit_price": round(exit_price, 2),
        "gross_pnl": round(gross, 2),
        "charges": round(fees["total_charges"], 2),
        "net_pnl": round(net, 2),
        "exit_reason": exit_reason,
        "buy_turnover": round(buy_turnover, 2),
        "sell_turnover": round(sell_turnover, 2),
        "cash_capital": round(capital, 2),
        "leverage": leverage,
        "notional_at_entry": round(entry_price * qty, 2),
    }

# =========================
# BACKTEST
# =========================
def run_backtest(cfg: Config) -> pd.DataFrame:
    all_trades = []

    for ticker in cfg.tickers:
        print(f"Downloading {ticker} ...")
        raw = fetch_5m_dataframe(ticker, cfg.start_date, cfg.end_date, cfg.interval, cfg.timezone)
        if raw.empty:
            print(f"  WARN: No data for {ticker}")
            continue

        df = compute_vwap_intraday(raw, cfg.timezone)

        # Unique list of session dates
        dates = sorted(list({pd.Timestamp(ts).date() for ts in df.index}))

        for d in dates:
            day = pd.Timestamp(d)
            day_df = session_filter(df, day, cfg.timezone)
            if len(day_df) < 10:
                continue

            # Find the first valid VWAP 3-candle entry
            direction, en_ts, en_price = find_vwap_entry(
                day_df=day_df,
                entry_start=cfg.entry_start,
                tz=cfg.timezone,
                enable_longs=cfg.enable_longs,
                enable_shorts=cfg.enable_shorts,
                max_vwap_gap_pct=cfg.max_vwap_gap_pct
            )
            if direction is None or en_ts is None or not np.isfinite(en_price):
                continue

            # Do not open too late
            last_entry_ts = _ts_on_day(day, cfg.entry_end, cfg.timezone)
            if en_ts > last_entry_ts:
                continue

            trade = simulate_trade(
                day_df=day_df, direction=direction,
                entry_ts=en_ts, entry_price=en_price,
                capital=cfg.capital_per_stock, leverage=cfg.intraday_leverage,
                sl_rupees=cfg.sl_rupees, tp_rupees=cfg.tp_rupees,
                squareoff_time=cfg.squareoff_time, tz=cfg.timezone,
                ticker=ticker,
                enable_trailing_sl=cfg.enable_trailing_sl,
                trail_rupees=cfg.trail_rupees
            )
            if trade:
                all_trades.append(trade)

    trades = pd.DataFrame(all_trades)
    if trades.empty:
        print("No trades generated.")
        return trades

    trades.sort_values(by=["date", "ticker", "entry_time"], inplace=True)
    trades.to_csv(cfg.out_file, index=False)
    return trades

# =========================
# METRICS
# =========================
def max_drawdown(series: pd.Series) -> float:
    cum = series.cumsum()
    peak = cum.cummax()
    dd = cum - peak
    return float(dd.min())

def summarize(trades: pd.DataFrame):
    print("\n=== OVERALL METRICS ===")
    n = len(trades)
    wins = (trades["net_pnl"] > 0).sum()
    win_rate = 100.0 * wins / n if n else 0.0
    gross = trades["gross_pnl"].sum()
    charges = trades["charges"].sum()
    net = trades["net_pnl"].sum()
    mdd = max_drawdown(trades["net_pnl"])

    print(f"Trades: {n} | Win rate: {win_rate:.1f}%")
    print(f"Gross P&L: ₹{gross:,.2f} | Charges: ₹{charges:,.2f} | Net P&L: ₹{net:,.2f}")
    print(f"Max Drawdown (net): ₹{mdd:,.2f}")

    print("\n=== BY TICKER ===")
    by_t = trades.groupby("ticker").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_t["win_rate"] = by_t["win_rate"].round(1)
    print(by_t.to_string(index=False))

    print("\n=== BY DIRECTION ===")
    by_dir = trades.groupby("direction").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_dir["win_rate"] = by_dir["win_rate"].round(1)
    print(by_dir.to_string(index=False))

    print("\nWrote trades to:", CFG.out_file)

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    trades = run_backtest(CFG)
    if not trades.empty:
        summarize(trades)


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYSKFIN.NS"}}}

1 Failed download:
['DUMMYSKFIN.NS']: YFTzMissingError('possibly delisted; no timezone found')


  WARN: No data for DUMMYSKFIN.NS


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYTATAM.NS"}}}

1 Failed download:
['DUMMYTATAM.NS']: YFTzMissingError('possibly delisted; no timezone found')


  WARN: No data for DUMMYTATAM.NS



1 Failed download:
['DUMMYDBRLT.NS']: YFTzMissingError('possibly delisted; no timezone found')


  WARN: No data for DUMMYDBRLT.NS

=== OVERALL METRICS ===
Trades: 9765 | Win rate: 40.2%
Gross P&L: ₹3,327,417.66 | Charges: ₹2,189,326.38 | Net P&L: ₹1,138,091.69
Max Drawdown (net): ₹-490,870.39

=== BY TICKER ===
       ticker  n  wins     gross  charges       net  win_rate
    360ONE.NS 20     9   8841.42  4489.40   4352.02      45.0
   3MINDIA.NS 20     7 -11575.00  4387.46 -15962.46      35.0
 AADHARHFC.NS 19     8   1695.38  4268.17  -2572.79      42.1
  AARTIIND.NS 20     6 -17759.37  4488.59 -22247.96      30.0
     AAVAS.NS 19     7   7472.48  4260.04   3212.43      36.8
       ABB.NS 20     8    552.50  4474.82  -3922.32      40.0
ABBOTINDIA.NS 21     7  -5200.00  4554.03  -9754.03      33.3
 ABCAPITAL.NS 21     9   7848.17  4718.92   3129.25      42.9
     ABFRL.NS 19    13  39178.17  4266.29  34911.88      68.4
     ABLBL.NS 20     7  -4132.16  4489.56  -8621.72      35.0
     ABREL.NS 19    13  50997.57  4267.60  46729.98      68.4
   ABSLAMC.NS 19     5   -465.14  4263.